# São Paulo Real Estate - Sale/Rent - April 2019

**D2APR: Aprendizado de Máquina e Reconhecimento de Padrões** (IFSP, Campinas)
Especialização em Ciência de Dados - IFSP Campinas

Alunos
- Daniel Vargas Shimamoto
- Diego Machado de Assis

## 1. Informações Gerais

### 1.1. Contexto do problema

### 1.2. Conhecendo  o dataset

O [Dataset](https://www.kaggle.com/argonalyst/sao-paulo-real-estate-sale-rent-april-2019) contem cerca de 13000 dados referentes a venda e aluguel de apartamentos na cidade de São Paulo (Brasil). Os dados foram coletados de diversas fontes, principalmente de sites de classificados de imóveis. Todos os dados foram coletados no mês de abril de 2019.


### 1.3. Principais atributos e seus tipos

Os dados dos imóveis possuem 16 atributos

* Price (int): Preço total anunciado em reais
* Condo (int): Condomínio em reais (Valores desconhecidos são marcados como zero)
* Size (int): Tamanho da propriedade em m² (Somente áreas privadas)
* Rooms (int): Número de Quartos
* Toilets (int): Númerto total de banheiros 
* Suites (int): Número de suites (Quartos com banheiros privativos)
* Parking (int): Número de vagas de estacionamento
* Elevator (binario): Se existe elevador (1 - sim, 0 - não)
* Furnished (binario): Se o imóvel é mobiliado (1 - sim, 0 - não)
* Swimming Pool (binario): Se existe piscina na propriedade (1 - sim, 0 - não)
* New (binario): Se o apartamento é novo (1 - sim, 0 - não) 
* District (string) : Bairro e cidade que o imóvel está localizado
* Negotiation Type (string): Alugel ou venda
* Property Type (string): Tipo de propriedade
* Latitude (float): Latitude do imóvel
* Longitude (float): Longitude do imóvel 

### 1.4 Objetivo

O Objetivo desse estudo é prever o valor dos apartamentos (*Price*) com base nas suas características, possibilitando a automatização desse processo. A criação de um modelo pode reduzir os custos de cotação realizado por empresas para inferir um valor aos imóveis.

## 2. Limpeza da base e Análise exploratória

### 2.1 Limpeza dos dados

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/kaggle/input/sao-paulo-real-estate-sale-rent-april-2019/sao-paulo-properties-april-2019.csv')

print(f'O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas')

df.head()

In [3]:
df.info()

In [4]:
# Dados nulos
print(f'O dataset possui {df.isnull().sum().sum()} dados Nulos')

In [5]:
# Dados duplicados
print(f'O dataset possui {df.duplicated().sum()} dados duplicados')

In [6]:
df[df.duplicated(keep=False)].sort_values(by='Price')

In [7]:
df.drop_duplicates(inplace=True)
print(f'Após a exlusão dos duplicados o dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas')

In [8]:
# Convertendo espaço por "_" no nome das colunas
df.columns = df.columns.str.replace(' ','_')
df.head(3)

### 2.2. Separação da base de teste

Após a limpeza dos dados duplicados e dos valores vazios, vamos começar a análise exploratória. Antes de começar a análisar os dados, vamos criar uma cópia do dataset limpo e dividir a base em dados de treino em teste. Para montar um modelo que seja consistente os dados de testes não devem ser usados para análisar e preparar a base, de modo que a divisão deve ser feita anteriormente.
O parâmetro **Negotiation_Type** define se o imóvel é para *venda* ou *aluguel* e ele afeta muito a variável alvo *Price*. Dessa forma vamos estratificar a amostra nesse parâmetro, garantindo a homogeneidade em relação a esse atributo.


In [9]:
new_df = df.copy()

new_df.head(3)

In [10]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(new_df, test_size=0.2, stratify=df['Negotiation_Type'], random_state=13)

print(f'Temos {df_train.shape} dados de treino e {df_test.shape} dados de teste')

In [11]:
df_train['Negotiation_Type'].value_counts()/len(df_train['Negotiation_Type'])

In [12]:
df_test['Negotiation_Type'].value_counts()/len(df_test['Negotiation_Type'])

### 2.3. Análise exploratória

#### 2.3.1. Variáveis Númericas

In [13]:
num_atri = ['Price','Condo','Size','Latitude','Longitude','Rooms','Toilets','Suites','Parking']
df_train[num_atri]

In [14]:
df_train[num_atri].describe().round(2)

In [15]:
df_train[num_atri].hist(bins=30, figsize=(15,10))
display()

In [16]:
fig, ax = plt.subplots(1, 3, figsize=(30, 6))

sns.boxplot(data = df_train, x='Price', ax = ax[0])
ax[0].set_title('Price - Boxplot', fontsize = 20)

sns.boxplot(data = df_train, x='Condo', ax = ax[1])
ax[1].set_title('Condo - Boxplot', fontsize = 20)

sns.boxplot(data = df_train, x='Size', ax = ax[2])
ax[2].set_title('Size - Boxplot', fontsize = 20)

As variáveis **Price**, **Condo** e **Size** possuem formato de uma curva assimétrica à esquerda. A análise dos boxplot dessas variáveis apontam vários outliers, porém nenhum deles é muito distante dos valores. Optamos por não excluir nenhum dos valores

A **Latitude** e **Longitude** possui uma distribuição muito concentrada e alguns valores zerados que podem ser interpretados como falta de informação das variáveis. 

**Rooms**, **Toilets**, **Suites** e **Parking** são variáveis que possuem uma moda bem evidente e támbem se assemelham a uma curva assimétrica à esquerda, porém com uma quantidade reduzida de valores. Os seus valores mínimos e máximos estão dentro dos valores esperados.

#### 2.3.2. Variáveis Categóricas

In [17]:
cat_atri = ['Elevator', 'Furnished', 'Swimming_Pool', 'New', 'District', 'Negotiation_Type', 'Property_Type']
df_train[cat_atri].head(3)

In [18]:
for i in cat_atri:
    print(i)
    print(f'{df_train[i].value_counts()}\n')

In [19]:
print(f'Existem {len(df_train["District"].unique())} valores distintos de "District" no dataset')

In [20]:
sns.boxplot(data = df_train, x='Price',y ='Negotiation_Type')

**Elevator**,**Furnished**,**Swimming Pool**,**New** são variáveis binárias, ou seja, os valores são 0 (negativo) ou 1 (positivo). 

A variável **Negotiation Type** também possui dois valores e será transformada em binária posteriormente. Um fato interessante sobre essa variável é que dependendo do tipo a variável alvo possui valores bem diferentes. O Range de preço para aluguel é bem inferior ao de venda.

Dentro de **District** existem 96 bairros diferentes, sendo Moema com maior número de ocorrências (238) e Perus o de menor valor (13)

A Coluna **Property Type** possui um valor único, visto que todos os imóveis da base são apartamentos

### 2.4. Negotiation Type = Rent

A partir de agora vamos dividir a análise exploratória em duas partes, a primeira será referente a apartamentos para alugar e a segunda para compra. Visto que a variável alvo é muito dependente do tipo de negociação do imóvel, vamos fazer análises separadas para encontrar as principais variáveis correlacionadas e como está a distribuição das demais

In [21]:
df_rent = df_train.query('Negotiation_Type == "rent"')
df_rent.head(3)

In [22]:
df_rent[num_atri].describe().round(2)

In [23]:
plt.figure(figsize=(10, 10))

mask = np.zeros_like(df_rent[num_atri].corr())
mask[np.triu_indices_from(mask, k=1)] = True

sns.heatmap(df_rent[num_atri].corr(), mask=mask, linewidths=.3, cmap=sns.diverging_palette(20, 220, as_cmap=True), vmin=-1, vmax=1, annot = True, fmt = '.2f')

In [24]:
sns.pairplot(df_rent, y_vars=["Price"], x_vars = num_atri)

Os apartamentos para alugar possuem uma alta correlação com as características internas do imóvel (Tamanho, quartos, banheiros, suites, estacionamentos). O valor do condomínio também possui uma correlação positiva com o preço do imóvel.

In [25]:
df_rent.groupby('Elevator')['Price'].describe().T.round(2)

In [26]:
df_rent.groupby('Furnished')['Price'].describe().T.round(2)

In [27]:
df_rent.groupby('Swimming_Pool')['Price'].describe().T.round(2)

In [28]:
df_rent.groupby('New')['Price'].describe().T.round(2)

In [29]:
df_rent.groupby('District')['Price'].describe()[['count','mean']].sort_values(by='mean', ascending = False)

Em relação as variáveis categóricas, temos que apartamentos mobiliados e com piscina possuem um preço mais caro. O Elevador não possui grande relação com o valor do aluguel. Sobre a condição do apartamento, temos poucas amostras de *apartamentos novos para alugar*, de modo que não é possível fazer essa comparação.

É possível encontrar uma grande diferença no aluguel em relação aos bairros, lugares consideramos mais "nobres" como Itaim Bibi e Alto de Pinheiros possuem um aluguel médio mais alto em relação a Itaim Paulista e Grajaú

### 2.5. Negotiation Type = Sale


In [30]:
df_sale = df_train.query('Negotiation_Type == "sale"')
df_sale.head(3)

In [31]:
df_sale[num_atri].describe().round(2)

In [32]:
plt.figure(figsize=(10, 10))

mask = np.zeros_like(df_sale[num_atri].corr())
mask[np.triu_indices_from(mask, k=1)] = True

sns.heatmap(df_sale[num_atri].corr(), mask=mask, linewidths=.3, cmap=sns.diverging_palette(20, 220, as_cmap=True), vmin=-1, vmax=1, annot = True, fmt = '.2f')

In [33]:
sns.pairplot(df_sale,y_vars=["Price"], x_vars = num_atri)

Os apartamentos para venda possuem uma alta correlação com as características internas do imóvel (Tamanho, quartos, banheiros, suites, estacionamentos). O valor do condomínio também possui uma correlação positiva com o preço do imóvel.

In [34]:
df_sale.groupby('Elevator')['Price'].describe().T.round(2)

In [35]:
df_sale.groupby('Furnished')['Price'].describe().T.round(2)

In [36]:
df_sale.groupby('Swimming_Pool')['Price'].describe().T.round(2)

In [37]:
df_sale.groupby('New')['Price'].describe().T.round(2)

In [38]:
df_sale.groupby('District')['Price'].describe()[['count','mean']].round(2).sort_values(by='mean', ascending = False)

Em relação as variáveis categóricas, temos que apartamentos mobiliados, com piscina e elevador possuem um preço mais caro.  Sobre a condição do apartamento, temos poucas amostras de *apartamentos novos para comprar*, de modo que não é possível fazer essa comparação.

É possível encontrar uma grande diferença no preço de venda dos apartamentos em relação aos bairros, lugares consideramos mais "nobres" como Iguatemi e Alto de Prinheiros possuem um valor médio de venda mais alto em relação a Cidade Tiradentes e Lajeado

### 2.6. Conclusão

O preço do imóvel possui uma alta correlação com as características do imóvel e conforme o aumento da quantidade de cômodos e atributos (estacionamento, piscina, mobilia) o preço tende a aumentar. A base de dados possui um atributo fundamental para definir o preço final, se o imóvel está para vender ou alugar. Um atributo bem importante é o bairro onde o imóvel está localizado, possuindo um preço médio diferente dependendo do local. É possível que a divisão da base tenha ocultado algum bairro e quando submetidos a dados não vistos venha ocorrer alguma inconsistência. Vamos tratar essa possibilidade mais a diante

Algumas colunas possuem uma relação menor com os dados. A latitude e longitude são numericamente pouco significante, podendo ser substituida pela informação do bairro em que o imóvel está localizado. A coluna Property_Type não oferece nenhuma informação adiconal pois possui um valor único. Por fim, pouco se consegue extrair da informação sobre o imóvel ser novo ou não pois a quantidade de imóveis novos são bem inferiores quando comparadas com usados.

## 3. Preparação das Bases

In [39]:
df_prep = df_train.copy()

### 3.1. Exclusão de colunas e divisão da base de dados

Vamos remover algumas colunas que não favorecem a criação do modelo. São elas 
* Latitude
* Longitude
* Property_Type

In [40]:
df_prep = df_prep.drop(columns = ['Property_Type','Latitude','Longitude'])
df_prep.head(3)

### 3.2. Transformação de variáveis

Temos duas variáveis que estão em formato de texto 
* District
* Negotiation_Type

Vamos transformar esses valores em númericos para facilitar a criação do modelo

#### 3.2.2. Negotiation_Type

* Sale → 1
* Rent → 0

In [41]:
Negotiation = {'sale':1,'rent':0}
df_prep['Negotiation_Type'] = df_prep['Negotiation_Type'].map(Negotiation)
df_prep.head(3)

#### 3.2.2. District

A variável *district* é uma variável categórica e vamos fazer o tratamento por meio do *one hot encoding* para transformar em colunas binárias. Essa transformação faz com que cada uma dos valores dessa feature seja transformada em uma coluna binária, onde é preenchida com o valor de **1** o valor verdadeiro e **0** as demais colunas. Para tratar a possibilidade de algum valor de *district* não estar na base de treino e apareça nos testes, vamos usar o parâmetro *handle_unknown = ignore*, de modo que caso isso ocorra, todas as colunas referentes terão o valor **0**.


In [42]:
#df_train['District'] = df_train['District'].map(district['index'])
# one hot encoding by pandas
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
df_prep_OHE = encoder.fit_transform(df_prep[['District']])

In [43]:
df_prep_OHE

In [44]:
encoder.categories_

### 3.3. Feature Scaling

As variáveis númericas possuem dimensões muito diferentes. Desse modo é interessante fazer uma transformação para que as unidades não possuam um intervalo de dados muito distinto. Como vimos na análise exploratória, os atributos **Condo** e **Size** possuem vários outliers. Dessa forma optamos por utilizar o *Z-Score Normalization* para transformar as variáveis.

In [45]:
from sklearn.preprocessing import StandardScaler

num_atri = ['Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking']

scaler = StandardScaler()

df_prep[num_atri] = scaler.fit_transform(df_prep[num_atri])
df_prep[num_atri]

In [46]:
df_prep[num_atri].hist(bins=30, figsize=(15,10))
display()

### 3.4. Pipeline de pré-processamento

Para facilitar o tratamento dos dados, vamos criar um *Pipeline* que agrupa todas as transformações que fizemos nos dados

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [48]:
drop_attributes = ['Property_Type', 'Latitude', 'Longitude']
numerical_attributes = ['Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking']
categorical_attributes = ['District']
binary_attributes = ['Elevator', 'Furnished', 'Swimming_Pool', 'New']

prep_pipeline = ColumnTransformer([
    ('drop_features', 'drop', drop_attributes),
    ('scale_numerical', StandardScaler(), numerical_attributes),
    ('encode_categorical', OneHotEncoder(handle_unknown='ignore'), categorical_attributes),
    ('keep_features', 'passthrough', binary_attributes)
], remainder='drop')

## 4. Modelos de Aprendizado de Máquina

Agora iremos criar alguns modelos de aprendizado de máquina, utilizando diversos algorítmos e hiperparâmetros, e comparar seus resultados. Isso será feito com auxílio da classe **GridSearchCV** do *sklearn*.

Optamos por treinar modelos distintos para os dados de *venda* e *aluguel*, uma vez que tratam-se de negociações distintas. Um mesmo imóvel, dependendo do tipo de negociação a que está submetido, poderá ter um preço de anúncio completamente diferente.

### 4.1. Separação dos dados de treinamento

Primeiramente vamos separar o conjunto de dados de treinamento em dois outros conjuntos, para **sale** e **rent**.

In [49]:
df_sale = df_train[df_train.Negotiation_Type == 'sale'].copy()
df_rent = df_train[df_train.Negotiation_Type == 'rent'].copy()

Para cada tipo de negociação, precisamos separar as variáveis independentes (*features*) da variável dependente, que queremos predizer (*target*).

In [50]:
df_sale_features = df_sale.drop(columns=['Price'])
df_sale_target = df_sale['Price'].values

df_rent_features = df_rent.drop(columns=['Price'])
df_rent_target = df_rent['Price'].values

Vamos criar um vetor de tuplas com informações de cada tipo de negociação, que será conveninente para execução dos modelos.

In [51]:
negotiations = [
    ('Sale', df_sale_features, df_sale_target),
    ('Rent', df_rent_features, df_rent_target)
]

### 4.2. Métrica de avaliação dos modelos

Para avaliar os modelos, vamos utilizar a **raiz dos erros médios ao quadrado** (*Root Mean Squared Error (RMSE)*). Para isso, criamos uma função auxiliar que recebe a instância do *GridSearchCV* e retorna, para cada conjunto de parâmetros, o valor do RMSE e seu desvio padrão.

In [52]:
import re

def gridsearch_rmse(gridsearchcv, name):
    rmse_scores = np.array(
        [np.sqrt(-value) for key, value in gridsearchcv.cv_results_.items() if re.match(r'split\d+_test_score', key)]
    )
    
    rmse_dict = {
        "Parameters": gridsearchcv.cv_results_['params'],
        "RMSE": np.mean(rmse_scores, axis=0),
        "std": np.std(rmse_scores, axis=0)
    }
    
    return pd.DataFrame(rmse_dict).style.set_caption(f'<b>Negotiation type: {name}</b>')\
                                        .format(precision=2, thousands=",")\
                                        .set_properties(**{'background-color': '#ffffb3'}, subset=gridsearchcv.best_index_)

### 4.3. Treinamento dos modelos

Para realizar o treinamento dos modelos iremos utilizar a classe **GridSearchCV** do pacote *sklearn.model_selection*. Será realizada validação cruzada para avaliação do modelo, utilizando o **Kfold** com 5 grupos.

A estrutura geral de execução de todos os modelos será a mesma. Inicialmente, criamos um *Pipeline* que executa o pipeline de preprocessamento definido anteriormente e, na sequência, a função de regressão específica. Definimos também um dicionário de parâmetros a serem testados pela função de *grid search*. Então, para cada tipo de negociação (*sale*, *rent*), executamos o `GridSearchCV` com os parâmetros especificados. O retorno do `GridSearchCV` é passado para a função auxiliar que criamos `gridsearch_rmse` que gera o resumo dos resultados da regressão.

In [53]:
from sklearn.model_selection import GridSearchCV

num_folds = 5

Vamos salvar os melhores modelos para futuro uso nos dados de teste

In [54]:
best_models = pd.DataFrame({
    'Sale': [],
    'Rent': []
})

#### 4.2.1. Regressão Linear

O primeiro modelo que criamos é de Regressão Linear. Esse é um modelo simples, com poucas opções de hiperparâmetros. Dessa forma, a única variação que testaremos será de uso do intercepto ($\theta_0$) no cálculo da regressão ou não.

In [55]:
from sklearn.linear_model import LinearRegression

linreg_pipeline = Pipeline([
    ('preprocessing', prep_pipeline),
    ('linear_regression', LinearRegression())
])

linreg_param_grid = {
    'linear_regression__fit_intercept': [True, False],
}

for (name, features, target) in negotiations:
    linreg_gridsearch = GridSearchCV(linreg_pipeline, linreg_param_grid, cv=num_folds, scoring='neg_mean_squared_error', return_train_score=True)
    linreg_gridsearch.fit(features, target)
    
    best_models.loc['linreg', name] = linreg_gridsearch.best_estimator_
    
    display(gridsearch_rmse(linreg_gridsearch, name))

Conforme destacado nos resultados acima, o melhor modelo de regressão linear calculado para venda de casas apresentou um erro de $R\$ 323.720\mathrm{,}45 \pm 20.560\mathrm{,}84$ e, para os alugueis, o erro do melhor modelo foi de $R\$ 2.029\mathrm{,}37 \pm 198\mathrm{,}21$.

O uso ou não do intercepto na regressão teve pouca influência sobre o erro do modelo. No caso de vendas, inclusive, os resultados foram iguais.

#### 4.2.2. Árvore de Decisão

Vamos testar agora um modelo de árvore de decisão. Este modelo permite a definição de diversos hiperparâmetros, sendo uma boa oportunidade para estudarmos o impacto de diferentes combinações no nosso erro calculado.

O valor de **max_depth** define a profundidade máxima da árvore de decisão criada. O padrão de `None` faz com que os nós sejam expandidos até as folhas se tornarem singulares ou contiverem menos que `min_samples_split` registros.

**min_samples_split** é o número mínimo de registros necessários para a divisão de um nó interno da árvore.

**max_features** define o número de variáveis considerado para se analizar a melhor divisão de um nó. O valor padrão de `auto` considera o total de variáveis do *dataset*. Os valores `sqrt` e `log2` consideram, respectivamente a raiz quadrada do total de variáveis e $\log_2$ total de variáveis.

Por fim, vamos considerar valores para **min_impurity_decrease**, que representa quanto o valor de *impureza* da árvore deve ser reduzido para que uma divisão de nó seja considerada.

Existem outros hiperparâmetros que poderiam ser ajustados no modelo, mas para evitar uma combinação muito extensa de busca do *grid search*, vamos nos deter a estes quatro citados.

In [56]:
from sklearn.tree import DecisionTreeRegressor

dectree_pipeline = Pipeline([
    ('preprocessing', prep_pipeline),
    ('decision_tree', DecisionTreeRegressor(random_state=50))
])

dectree_param_grid = {
    'decision_tree__max_depth': [None, 10, 100],
    'decision_tree__min_samples_split': [2, 5, 10, 50],
    'decision_tree__max_features': ['auto', 'sqrt', 'log2'], 
    'decision_tree__min_impurity_decrease': [0.0, 1.0, 2.0, 10.0, 50.0],
}

for (name, features, target) in negotiations:
    dectree_gridsearch = GridSearchCV(dectree_pipeline, dectree_param_grid, cv=num_folds, scoring='neg_mean_squared_error', return_train_score=True)
    dectree_gridsearch.fit(features, target)
    
    best_models.loc['dectree', name] = dectree_gridsearch.best_estimator_

    display(gridsearch_rmse(dectree_gridsearch, name))

Nos registros de **sale**, o melhor modelo foi treinado com o conjunto de parâmetros de `max_depth = None`, `max_features = 'auto'`, `min_impurity_decrease = 50.0` e `min_samples_split = 10`, apresentando um RMSE de $R\$312.428\mathrm{,}48 \pm 43.860\mathrm{,}48$.

Para os registros de **rent**, tivemos como melhores hiperparâmetros os valores de `max_depth = None`, `max_features = 'auto'`, `min_impurity_decrease = 50.0` e `min_samples_split = 50`, com um RMSE de $R\$2.132\mathrm{,}67 \pm 262\mathrm{,}72$

#### 4.2.3. Random Forest

In [57]:
from sklearn.ensemble import RandomForestRegressor

randforest_pipeline = Pipeline([
    ('preprocessing', prep_pipeline),
    ('random_forest', RandomForestRegressor(random_state=50))
])

randforest_param_grid = [
    {
        'random_forest__n_estimators': [10, 100],
        'random_forest__min_samples_split': [2, 10],
        'random_forest__min_impurity_decrease': [0.0, 50.0]
    },
    {
        'random_forest__n_estimators': [1000],
        'random_forest__max_samples': [0.5]
    }
]

for (name, features, target) in negotiations:
    randforest_gridsearch = GridSearchCV(randforest_pipeline, randforest_param_grid, cv=num_folds, scoring='neg_mean_squared_error', return_train_score=True)
    randforest_gridsearch.fit(features, target)
    
    best_models.loc['randforest', name] = randforest_gridsearch.best_estimator_
    
    display(gridsearch_rmse(randforest_gridsearch, name))

### 4.3. Avaliação dos modelos no conjunto de teste

Vamos executar os melhores modelos treinados para cada algoritmo utilizado nos dados de teste.

In [58]:
df_test_sale = df_test[df_test.Negotiation_Type == 'sale'].copy()
df_test_rent = df_test[df_test.Negotiation_Type == 'rent'].copy()

X_sale = df_test_sale.drop(columns=['Price'])
y_sale = df_test_sale['Price'].values

X_rent = df_test_rent.drop(columns=['Price'])
y_rent = df_test_rent['Price'].values

In [59]:
from sklearn.metrics import mean_squared_error

fig, axs = plt.subplots(len(best_models), 3, figsize=(22, 12))

i = 0
for model_name, (sale_model, rent_model) in best_models.iterrows():
    print(f'Model: {model_name}')
    
    pred_sale = sale_model.predict(X_sale)
    pred_rent = rent_model.predict(X_rent)
    
    rmse_sale = mean_squared_error(y_sale, pred_sale, squared=False)
    rmse_rent = mean_squared_error(y_rent, pred_rent, squared=False)
    
    y_test = np.append(y_sale, y_rent)
    pred_model = np.append(pred_sale, pred_rent)
    rmse_model = mean_squared_error(y_test, pred_model, squared=False)
    
    print(f' Sale RMSE: {rmse_sale}')
    print(f' Rent RMSE: {rmse_rent}')
    print(f' Model RMSE: {rmse_model}\n')
    
    sns.scatterplot(x=pred_model, y=y_test, ax=axs[i][0])
    axs[i][0].set_title(f'{model_name}: Prediction vs Real')
    axs[i][0].set_xlabel('Prediction')
    axs[i][0].set_ylabel('Real')

    residual = y_test - pred_model
    sns.scatterplot(x=pred_model, y=residual, ax=axs[i][1])
    axs[i][1].set_title(f'{model_name}: Prediction vs Residual')
    axs[i][1].set_xlabel('Prediction')
    axs[i][1].set_ylabel('Residual')
    
    sns.histplot(residual, ax=axs[i][2])
    axs[i][2].set_title(f'{model_name}: Residual distribution')
    axs[i][2].set_xlabel('Residual')
    axs[i][2].set_ylabel('Frequency')
    
    i += 1

plt.suptitle("Error visualization", fontsize = 22)
plt.show()

### 4.3. Discussão dos resultatos
* Métricas utilizadas
* Há overfitting ou underfitting?

### 4.5. Próximos passos
* Estratégias, ideias, sugestões para melhorar o modelo